<a href="https://colab.research.google.com/github/iqfareez/DLWorkshop-IIUM/blob/main/Assignments/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruction

**Assignment 2** – Submission Deadline: 30/09/2022 11:59PM MYT

In our session, we trained Resnet using the fruit dataset and achieved 83% accuracy. On the fruit dataset, you are required to prepare one notebook that can run on google collab containing:

1. The training of the model on the dataset after modification to improve the performance of the
model. *(5M)*
1. The training of the model on the dataset after data-centric strategies.*(5M)*
1. Save the final model and load again on the notebook. Perform inference on two new samples (fruit data captured from your phone). *(2Marks)*

\** the model can be any model, custom or your own defined model

\* the deliverables: one notebook containing all the codes, model file (.pt file) and inference
images